In [1]:
import ast
import os
import sys
import time

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
project_home = os.path.join("../")
sys.path.append(project_home)
data_path = os.path.join(project_home, "data/merged_preprocessed.csv")

In [3]:
from lib.apriori import runApriori

In [4]:
df = pd.read_csv(data_path, encoding="latin1")

In [5]:
len(df)

375197

In [6]:
df.head()

,college_name,levell,programme,discipline_group,discipline,type,year,total_general_total,total_general_females,total_backward_castes_total,...,other_minority_general_females,other_minority_backward_castes_total,other_minority_backward_castes_females,other_minority_total_persons,other_minority_total_females,state,city,speciality,girl_exclusive,student_hostel_available
0,"SMT. BHUDEVI MAHAVIDYALAYA SHASTRI PURAM, AGRA",Under Graduate,B.Sc.-Bachelor of Science,Science,science,Self Financing,3.0,0.306122,0.126531,0.693878,...,0.0,0.0,0.0,0.0,0.0,Uttar Pradesh,AGRA,0,False,False
1,Techno India 130,Under Graduate,B.Tech.-Bachelor of Technology,Other Engineering & Technology,food technology(ug),Self Financing,2.0,1.000000,0.200000,0.000000,...,0.0,0.0,0.0,0.0,0.0,West Bengal,KOLKATA,Engineering & Technology,False,False
2,"Shri D N Institute of Computer Applications, A...",Under Graduate,B.C.A.-Bachelor of Computer Applications,Computer Application,computer science,Self Financing,3.0,0.808824,0.183824,0.191176,...,0.0,0.0,0.0,0.0,0.0,Gujarat,Anand-388001,Computer Application,False,True
3,Techno India 309,Under Graduate,B.H.M.-Bachelor of Hotel Management,Hotel Management,bhm,Self Financing,3.0,1.000000,0.271186,0.000000,...,0.0,0.0,0.0,0.0,0.0,West Bengal,KOLKATA,0,False,False
4,Sigma Institute of Physiotherapy,Under Graduate,B.P.T.-Bachelor of Physiotherapy,Physiotherapy,physiotherapy,Self Financing,3.0,0.605263,0.578947,0.394737,...,0.0,0.0,0.0,0.0,0.0,Gujarat,Vadodara - 390019,Physiotherapy,False,True


In [7]:
df.columns

Index(['college_name', 'levell', 'programme', 'discipline_group', 'discipline',
       'type', 'year', 'total_general_total', 'total_general_females',
       'total_backward_castes_total', 'total_backward_castes_females',
       'total_total_persons', 'total_total_females', 'pwd_general_total',
       'pwd_general_females', 'pwd_backward_castes_total',
       'pwd_backward_castes_females', 'pwd_total_persons', 'pwd_total_females',
       'muslim_minority_general_total', 'muslim_minority_general_females',
       'muslim_minority_backward_castes_total',
       'muslim_minority_backward_castes_females',
       'muslim_minority_total_persons', 'muslim_minority_total_females',
       'other_minority_general_total', 'other_minority_general_females',
       'other_minority_backward_castes_total',
       'other_minority_backward_castes_females',
       'other_minority_total_persons', 'other_minority_total_females', 'state',
       'city', 'speciality', 'girl_exclusive', 'student_hostel_availab

### Converting to required dataset format

In [8]:
columns_set_1 = ["levell", "programme", "total_general_total", "total_backward_castes_total", "pwd_total_persons", 
                "muslim_minority_total_persons", "other_minority_total_persons"
                ]
columns_set_2 = ["programme", "total_general_females", "total_backward_castes_females", "pwd_total_females",
                 "muslim_minority_total_females", "other_minority_total_females"
                ]

In [9]:
def binning(columns, df, bins=[(0,0.2), (0.2,0.5), (0.5,1.01)]):
    for column in columns:
        for bin_interval in bins:
            name = column + " [{}-{}]".format(bin_interval[0], bin_interval[1])
            df[name] = (df[column] >= bin_interval[0]) & (df[column] < bin_interval[1])
        df.drop(column, axis=1, inplace=True)
    
    return df

In [10]:
def one_hot_enc(column_name, df):
    d = pd.get_dummies(df[column_name])
    df = pd.concat([d, df], axis=1)
    df.drop(column_name, axis=1, inplace=True)
    return df

In [11]:
def convert_to_apriori_dataset(apriori_df, non_bool_cols = ["level", "programme"], 
                               bool_col_names=["general", "BC", "PWD", "muslim", "other"]):
    new_df = apriori_df[non_bool_cols]
    apriori_df.drop(columns=non_bool_cols, axis=1, inplace=True)
    new_col = apriori_df.apply(lambda x: apriori_df.columns[x].tolist(), axis=1)
    new_col = pd.DataFrame(new_col.tolist(), columns = bool_col_names)
    new_df = pd.concat([new_df, new_col], axis=1)
    
    return new_df

In [12]:
apriori_df = df[columns_set_1].copy()
apriori_df.columns = ["level", "programme", "general", "BC", "PWD", "muslim", "other"]

In [13]:
cols = ["general", "BC", "PWD", "muslim", "other"]
bins = [(0, 0.2), (0.2,0.5), (0.5,0.7), (0.7,1.01)]
apriori_df = binning(cols, apriori_df, bins)

In [14]:
apriori_df.head()

,level,programme,general [0-0.2],general [0.2-0.5],general [0.5-0.7],general [0.7-1.01],BC [0-0.2],BC [0.2-0.5],BC [0.5-0.7],BC [0.7-1.01],...,PWD [0.5-0.7],PWD [0.7-1.01],muslim [0-0.2],muslim [0.2-0.5],muslim [0.5-0.7],muslim [0.7-1.01],other [0-0.2],other [0.2-0.5],other [0.5-0.7],other [0.7-1.01]
0,Under Graduate,B.Sc.-Bachelor of Science,False,True,False,False,False,False,True,False,...,False,False,True,False,False,False,True,False,False,False
1,Under Graduate,B.Tech.-Bachelor of Technology,False,False,False,True,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False
2,Under Graduate,B.C.A.-Bachelor of Computer Applications,False,False,False,True,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False
3,Under Graduate,B.H.M.-Bachelor of Hotel Management,False,False,False,True,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False
4,Under Graduate,B.P.T.-Bachelor of Physiotherapy,False,False,True,False,False,True,False,False,...,False,False,True,False,False,False,True,False,False,False


In [15]:
non_bool_cols = ["level", "programme", "state", "speciality"]
bool_col_names = ["general", "BC", "PWD", "muslim", "other"]

In [16]:
%%time
apriori_df = convert_to_apriori_dataset(apriori_df)

CPU times: user 31.1 s, sys: 78.8 ms, total: 31.2 s
Wall time: 31.2 s


In [17]:
extra_df = df[["state", "speciality", "girl_exclusive", "student_hostel_available"]].copy()

In [18]:
extra_df["girl_exclusive"] = extra_df["girl_exclusive"].apply(lambda x: "Girl_exclusive" if x else "Co-ed")
extra_df["student_hostel_available"] = extra_df["student_hostel_available"].apply(
    lambda x: "Hostel_available" if x else "No_hostel"
)
extra_df["speciality"] = extra_df["speciality"].replace({'0': "No speciality"})

In [19]:
apriori_df = pd.concat([apriori_df, extra_df], axis=1)
del extra_df

In [20]:
apriori_df

,level,programme,general,BC,PWD,muslim,other,state,speciality,girl_exclusive,student_hostel_available
0,Under Graduate,B.Sc.-Bachelor of Science,general [0.2-0.5],BC [0.5-0.7],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Uttar Pradesh,No speciality,Co-ed,No_hostel
1,Under Graduate,B.Tech.-Bachelor of Technology,general [0.7-1.01],BC [0-0.2],PWD [0-0.2],muslim [0-0.2],other [0-0.2],West Bengal,Engineering & Technology,Co-ed,No_hostel
2,Under Graduate,B.C.A.-Bachelor of Computer Applications,general [0.7-1.01],BC [0-0.2],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Gujarat,Computer Application,Co-ed,Hostel_available
3,Under Graduate,B.H.M.-Bachelor of Hotel Management,general [0.7-1.01],BC [0-0.2],PWD [0-0.2],muslim [0-0.2],other [0-0.2],West Bengal,No speciality,Co-ed,No_hostel
4,Under Graduate,B.P.T.-Bachelor of Physiotherapy,general [0.5-0.7],BC [0.2-0.5],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Gujarat,Physiotherapy,Co-ed,Hostel_available
...,...,...,...,...,...,...,...,...,...,...,...
375192,Under Graduate,B.Sc.-Bachelor of Science,general [0-0.2],BC [0.7-1.01],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Uttar Pradesh,No speciality,Co-ed,No_hostel
375193,Under Graduate,M.B.B.S.-Bachelor of Medicine and Bachelor of ...,general [0.5-0.7],BC [0.2-0.5],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Haryana,Medical-Allopathy,Co-ed,Hostel_available
375194,Post Graduate,M.D.-Doctor of Medicine,general [0-0.2],BC [0-0.2],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Haryana,Medical-Allopathy,Co-ed,Hostel_available
375195,Under Graduate,B.A.-Bachelor of Arts,general [0.7-1.01],BC [0-0.2],PWD [0-0.2],muslim [0-0.2],other [0-0.2],Uttar Pradesh,No speciality,Co-ed,No_hostel


### Frequent pattern and rule gen

In [21]:
def data_generator(df):
    for idx in range(len(df)):
        record = frozenset(df.iloc[idx])
        yield record


In [22]:
data = data_generator(apriori_df)

In [23]:
data

<generator object data_generator at 0x7fdfb01fd1a8>

#### The below cell takes an average of 4.5 mins to run, skip to bottom of the notebook to get the same results quicker with the use of mlxtend library

In [24]:
%%time
items, rules = runApriori(data, minSupport=0.15, minConfidence=0.6)

CPU times: user 4min 51s, sys: 324 ms, total: 4min 52s
Wall time: 4min 52s


In [25]:
sorted(items, key=lambda x: x[1], reverse=True)

[(('PWD [0-0.2]',), 0.9980250375136261),
 (('other [0-0.2]',), 0.953288539087466),
 (('PWD [0-0.2]', 'other [0-0.2]'), 0.9514255177946519),
 (('muslim [0-0.2]',), 0.9317185371951269),
 (('PWD [0-0.2]', 'muslim [0-0.2]'), 0.9299941097610056),
 (('other [0-0.2]', 'muslim [0-0.2]'), 0.8899164972001375),
 (('PWD [0-0.2]', 'other [0-0.2]', 'muslim [0-0.2]'), 0.8882453750962828),
 (('Co-ed',), 0.885081703744966),
 (('PWD [0-0.2]', 'Co-ed'), 0.8833812637094647),
 (('other [0-0.2]', 'Co-ed'), 0.8465819289599864),
 (('PWD [0-0.2]', 'other [0-0.2]', 'Co-ed'), 0.8449561163868581),
 (('muslim [0-0.2]', 'Co-ed'), 0.8254676876414258),
 (('PWD [0-0.2]', 'muslim [0-0.2]', 'Co-ed'), 0.8239804689269904),
 (('other [0-0.2]', 'Co-ed', 'muslim [0-0.2]'), 0.7910564316878866),
 (('PWD [0-0.2]', 'muslim [0-0.2]', 'other [0-0.2]', 'Co-ed'),
  0.7896065267046378),
 (('Under Graduate',), 0.7460720634759873),
 (('PWD [0-0.2]', 'Under Graduate'), 0.7445661878959586),
 (('other [0-0.2]', 'Under Graduate'), 0.713545

In [26]:
sorted(rules, key=lambda x: x[1], reverse=True)

[((('BC [0.2-0.5]', 'other [0-0.2]', 'Co-ed', 'muslim [0-0.2]'),
   ('PWD [0-0.2]',)),
  0.9991928974979821),
 ((('BC [0.2-0.5]', 'muslim [0-0.2]', 'Under Graduate'), ('PWD [0-0.2]',)),
  0.9991800418702025),
 ((('BC [0.2-0.5]', 'muslim [0-0.2]', 'Co-ed'), ('PWD [0-0.2]',)),
  0.9991790677194502),
 ((('BC [0.2-0.5]', 'muslim [0-0.2]', 'other [0-0.2]'), ('PWD [0-0.2]',)),
  0.9990725908439423),
 ((('BC [0.2-0.5]', 'muslim [0-0.2]'), ('PWD [0-0.2]',)), 0.9990381828243169),
 ((('BC [0.2-0.5]', 'other [0-0.2]', 'Under Graduate'), ('PWD [0-0.2]',)),
  0.9990324551873981),
 ((('BC [0.2-0.5]', 'other [0-0.2]', 'Co-ed'), ('PWD [0-0.2]',)),
  0.9990202386523163),
 ((('BC [0.2-0.5]', 'Co-ed'), ('PWD [0-0.2]',)), 0.9989932145418436),
 ((('BC [0.2-0.5]', 'Under Graduate'), ('PWD [0-0.2]',)), 0.9989595338711743),
 ((('BC [0.2-0.5]', 'general [0.5-0.7]'), ('PWD [0-0.2]',)),
  0.9989566755083997),
 ((('general [0.5-0.7]', 'other [0-0.2]', 'muslim [0-0.2]'), ('PWD [0-0.2]',)),
  0.9989416520637784),
 

In [27]:
rules_list = []
for rule in rules:
    rules_list.append({
        "precedent": rule[0][0],
        "antecendent": rule[0][1],
        "confidence": rule[1]
    })
rules_df = pd.DataFrame(rules_list)

In [ ]:
rules_df.to_csv("../results/apriori_latest.csv", index=False)

### Lift calculation

In [28]:
full_set = apriori_df.values

In [29]:
super_set = []
for row in full_set:
    super_set.append(set(row))

In [30]:
del full_set

In [31]:
super_set

[{'B.Sc.-Bachelor of Science',
  'BC [0.5-0.7]',
  'Co-ed',
  'No speciality',
  'No_hostel',
  'PWD [0-0.2]',
  'Under Graduate',
  'Uttar Pradesh',
  'general [0.2-0.5]',
  'muslim [0-0.2]',
  'other [0-0.2]'},
 {'B.Tech.-Bachelor of Technology',
  'BC [0-0.2]',
  'Co-ed',
  'Engineering & Technology',
  'No_hostel',
  'PWD [0-0.2]',
  'Under Graduate',
  'West Bengal',
  'general [0.7-1.01]',
  'muslim [0-0.2]',
  'other [0-0.2]'},
 {'B.C.A.-Bachelor of Computer Applications',
  'BC [0-0.2]',
  'Co-ed',
  'Computer Application',
  'Gujarat',
  'Hostel_available',
  'PWD [0-0.2]',
  'Under Graduate',
  'general [0.7-1.01]',
  'muslim [0-0.2]',
  'other [0-0.2]'},
 {'B.H.M.-Bachelor of Hotel Management',
  'BC [0-0.2]',
  'Co-ed',
  'No speciality',
  'No_hostel',
  'PWD [0-0.2]',
  'Under Graduate',
  'West Bengal',
  'general [0.7-1.01]',
  'muslim [0-0.2]',
  'other [0-0.2]'},
 {'B.P.T.-Bachelor of Physiotherapy',
  'BC [0.2-0.5]',
  'Co-ed',
  'Gujarat',
  'Hostel_available',
  'P

In [ ]:
## Run this is running only for lift calc on stored results
results = pd.read_csv("../results/apriori_latest.csv")

In [32]:
## If results exist as a variable in notebook
results = rules_df

In [33]:
results.columns = ["antecedent", "consequent", "confidence"]

In [42]:
set(list(results["antecedent"][0])).issubset(super_set[1])

True

In [34]:
set(list(ast.literal_eval(results["antecedent"][0]))).issubset(super_set[1])

ValueError: malformed node or string: ('other [0-0.2]',)

In [43]:
def count_subsets(super_set, ant):
    c=0
    for s in super_set:
        if ant.issubset(s):
            c += 1
    return c

In [44]:
results["lift"] = None

In [45]:
def calc_lift(results, super_set):
    lift = []
    for row in tqdm(results.itertuples()):
        try:
            ant = set(list(ast.literal_eval(row.antecedent)))
        except:
            ant = set(list(row.antecedent))
        c = count_subsets(super_set, ant)
        lift.append(row.confidence / c)

In [ ]:
lift = []
for row in tqdm(results.itertuples()):
    try:
        ant = set(list(ast.literal_eval(row.antecedent)))
    except:
        ant = set(list(row.antecedent))
    c = count_subsets(super_set, ant)
    lift.append(row.confidence / c)

In [ ]:
results["lift"] = lift

In [ ]:
results["lift"] *= len(df)

In [ ]:
results.to_csv("../results/apriori_with_lift.csv", index=False)

### Verify results using mlxtend library

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
columns_set_1 = ["levell", "programme", "total_general_total", "total_backward_castes_total", "pwd_total_persons", 
                "muslim_minority_total_persons", "other_minority_total_persons",
                 "state", "speciality", "girl_exclusive", "student_hostel_available"
                ]

In [ ]:
apriori_df = df[columns_set_1].copy()
apriori_df.columns = ["level", "programme", "general", "BC", "PWD", "muslim", "other",
                     "state", "speciality", "girl_exclusive", "student_hostel_available"]

In [ ]:
# Only run this when using mlxtend library :
apriori_df = one_hot_enc("programme", apriori_df)
apriori_df = one_hot_enc("level", apriori_df)
apriori_df = one_hot_enc("state", apriori_df)
apriori_df = one_hot_enc("speciality", apriori_df)
apriori_df = one_hot_enc("girl_exclusive", apriori_df)
apriori_df = one_hot_enc("student_hostel_available", apriori_df)

In [ ]:
cols = ["general", "BC", "PWD", "muslim", "other"]
bins = [(0, 0.2), (0.2,0.5), (0.5,0.7), (0.7,1.01)]
apriori_df = binning(cols, apriori_df, bins)

In [ ]:
%%time
frequent_patterns = apriori(apriori_df, min_support=0.15, use_colnames=True)

In [ ]:
frequent_patterns.sort_values(by="support", ascending=False)

In [ ]:
%%time
assoc_rules = association_rules(frequent_patterns, metric="confidence", min_threshold=0.6)

In [ ]:
assoc_rules

In [ ]:
rules.to_csv("../association_rules_sup_0.1_conf_0.6.csv", index=False)